# ミニバッチ学習の実装

In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import math

# Load the MNIST dataset
import tensorflow as tf
mnist = tf.keras.datasets.mnist
(X_train, y_train),(X_test, y_test) = mnist.load_data()

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()

train = X_train/255
test = X_test/255
train = train.reshape(-1, 28*28)
test = test.reshape(-1, 28*28)
train_labels = lb.fit_transform(y_train)
test_labels = lb.fit_transform(y_test)

## ミニバッチ学習
* ミニバッチ学習は、一般的には非復元抽出によって行われることが多いが、必ずこうしなければならないというわけではなく、分析者がデータセットの与え方を工夫することもできる。ただし、工夫しても計算が上手くいくとは限らない。
* 工夫のしどころ。
    * 一般的には、エポックのたびにシャッフルするが、シャッフルするタイミングを任意に変えてみる  
    * 与えるミニバッチ の順番を意図的に操作してみる   
        * 例、出現頻度の少ないラベルのデータを先に学習させる
    * 抽出されるラベルの割合が一定になるように抽出してみる
    * 復元抽出にしてみる

In [2]:
def trainer(network, x, y):
    """
    学習用の関数
    このnotebookでは、ミニバッチ学習を学ぶことが目的であるため、この関数の中身は空のままにしておく
    実際には、何らかの学習させるための処理を記述する
    """
    pass
    return 

### ミニバッチ学習のループ(復元抽出)

In [3]:
np.random.seed(1234)
train_size = train_labels.shape[0]
batch_size = 32
max_iter = 10  #ループの回数
network = None #ダミー

for i in range(max_iter):
    batch_mask = np.random.choice(train_size, batch_size) # 復元抽出
    print("i=%s, "%i, "batch_mask=%s"%batch_mask[:10])
    x_batch = train[batch_mask]
    y_batch = train_labels[batch_mask]

    trainer(network, x_batch, y_batch)

i=0,  batch_mask=[27439 58067 34086 56373 23924 17048 55289 32399 55985 41239]
i=1,  batch_mask=[22267 21580 14629 12198 50682 46275 10983 23691 39552 21225]
i=2,  batch_mask=[29280 27532 28869 31741 49777  3039  8165 28319  8953 29692]
i=3,  batch_mask=[50926 18991  3062 43252 43382 58031 58255 51746    10  3356]
i=4,  batch_mask=[48917 55492 14455 34791 59642 43444 43456 20949  4468 43881]
i=5,  batch_mask=[12184 57310 45801 55823  6984  4994 52624   619 21634 19186]
i=6,  batch_mask=[27117 42059  4111 34580  4880 36288 26464 32382 26835 40249]
i=7,  batch_mask=[ 9624 10252 10700 18272  7688 13615 12057 51949 55061 35990]
i=8,  batch_mask=[49068 54819 35754 49556 43802 12633 59499 36759 32386 47848]
i=9,  batch_mask=[ 9532 31315 31088 29429 21129 37436 32946 35249 59498 17095]


### 復元抽出部分を理解するためのコード

In [4]:
np.random.seed(1234)
batch_mask = np.random.choice(train_size, batch_size)
print(len(set(batch_mask)))
print("batch_mask=", batch_mask)
print()

x_batch = train[batch_mask]
print("x_batch=", x_batch)
print("x_batch.shape=", x_batch.shape)
print()
y_batch = train_labels[batch_mask]
print("y_batch=", y_batch)
print("y_batch.shape=", y_batch.shape)
print()

32
batch_mask= [27439 58067 34086 56373 23924 17048 55289 32399 55985 41239  9449 23706
  8222 32427 33950 40684  8060  7962 13686 59834 59512 14192  7644 27973
 27984 41929 51583 49398  2558 36271 38450  3824]

x_batch= [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
x_batch.shape= (32, 784)

y_batch= [[0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0

In [7]:
# 復元抽出部分(何回か実行してみてください)
np.random.choice(10,3)

array([5, 4, 4])

### ミニバッチ学習のループ(非復元抽出)

### [演習]
* 以下の非復元抽出によるミニバッチ学習を完成させましょう。
* 通常の計算では、非復元抽出で行うことが多いです。

In [6]:
# ヒント
index = np.arange(10)
print("index=%s"%index)
np.random.seed(1234)
np.random.shuffle(index)
print("index=%s"%index)
print()
print(np.random.permutation(10))
print()

for i in range(4):
    print(index[3*i:3*(i+1)])
    
print(np.ceil(1.1), np.ceil(1.7), np.ceil(2.7)) # ceilは切り上げ関数

index=[0 1 2 3 4 5 6 7 8 9]
index=[7 2 9 1 0 8 4 5 6 3]

[7 3 5 1 4 8 0 2 6 9]

[7 2 9]
[1 0 8]
[4 5 6]
[3]
2.0 2.0 3.0


In [10]:
np.random.seed(1234)
train_size = train_labels.shape[0]
batch_size = 32
epochs = 10
network = None #ダミー
minibatch_num = np.ceil(train_size / batch_size).astype(int) # ミニバッチの個数
    
for epoch in range(epochs):
    print()
    
    # indexを定義し、シャッフルする
    index = np.arange(train_size)
    np.random.shuffle(index)
    
    for mn in range(minibatch_num):
        """
        非復元抽出によるループ
        """
        batch_mask = index[batch_size*mn:batch_size*(mn+1)]     
        print("epoch=%s, "%epoch, "batch_mask=%s"%batch_mask[:10])
        x_batch = train[batch_mask]
        y_batch = train_labels[batch_mask]

        trainer(network, x_batch, y_batch)


epoch=0,  batch_mask=[30329 44957 30866 40447 25580  6216 26373  9010 23445   108]
epoch=0,  batch_mask=[13638 33368 49673 31890  4891 31171 44103 57851   496  8184]
epoch=0,  batch_mask=[29174 33167 13319 44233 53744 55496 56873 40631 44358 58144]
epoch=0,  batch_mask=[40541 56435 12662   831 17775 10047  6291 23045 31666  2987]
epoch=0,  batch_mask=[36278 23185  8140 22416 51789 26671 22337 46976 22482 30227]
epoch=0,  batch_mask=[25895 28676 38529 48020 13975 59520  1465  7905 57265 33004]
epoch=0,  batch_mask=[26568 20475 43172 32824 23933 14617 39765 20889 19788 48315]
epoch=0,  batch_mask=[54250  6397 17608 21736 59230 13391 44222  5637 18489  9823]
epoch=0,  batch_mask=[49311 12279  4107 10231 37346 25028 12961 17640 47765 22661]
epoch=0,  batch_mask=[15830 26793 32555 47856 25701 13929 18850 28542  8736 28550]
epoch=0,  batch_mask=[ 9604 56237 46403 36689 41305 53030  3272 13921   806  8014]
epoch=0,  batch_mask=[56428 33449 23359 55130 10066  8925  2652 54683 42575   173]
epo

epoch=0,  batch_mask=[48154 12063 18204 49801 45995 10389 49130 36099 46904 41187]
epoch=0,  batch_mask=[49406  4819 22008 37715  5906 15061 15387 10623  3349 44129]
epoch=0,  batch_mask=[28009 48303 18227 33893  8895  8173 58700 43366 44612 29058]
epoch=0,  batch_mask=[24354  2414 37997 10907 28772  9571 47515 59931 17378 36637]
epoch=0,  batch_mask=[58072 19129 31307  6188 59393 11867 35761 25227 31881 12975]
epoch=0,  batch_mask=[33634 41012 12558  2723 43143 48755  5981 25025 52307   671]
epoch=0,  batch_mask=[11952 55560 16024 13069 36839  8159  8255 11485 54609 50577]
epoch=0,  batch_mask=[25344 56072 39966 43433 57894 53882 59493 13962  9152 31782]
epoch=0,  batch_mask=[10316 26035 27621  1354 37497 14804 12372 28188  7955 22351]
epoch=0,  batch_mask=[15020 37445  4535  9783 42447  6036 57772 10161  6264 41388]
epoch=0,  batch_mask=[25109  2110 46468 34555 24335 21702  7077 56011 37707 30979]
epoch=0,  batch_mask=[39377 45514 10006 11613 22309 19290  6158  9400 29601  4808]
epoc

epoch=1,  batch_mask=[13242 11171 34445 57481 28137 47499 24995 22556 26955 27404]
epoch=1,  batch_mask=[42769 48359 26027 39318 34522 47002 21689 38310 14116 37615]
epoch=1,  batch_mask=[24703 11723 33539 11398 43149 38620 47432 30171 16529  6248]
epoch=1,  batch_mask=[ 5545  7406  4895  1782 38176 33778 28709 24294  1870 33010]
epoch=1,  batch_mask=[ 8245 50336 26763 31625 45087 22735 15252 56999 31917 17600]
epoch=1,  batch_mask=[22779 41849 55063 45264 52009 21612 18551 40273 58122 56869]
epoch=1,  batch_mask=[23351 47335  8803  2219 35071 13301 55966 37069 51910 45953]
epoch=1,  batch_mask=[19034 39349 17686 44091 55997 49649 27794 21574 37826  4484]
epoch=1,  batch_mask=[30684 51603  1776 14953  8475  7842 40350 16606 38189 47751]
epoch=1,  batch_mask=[37507 49139 56078 57329 59645 49462 16588 25189 24378  7003]
epoch=1,  batch_mask=[24306  3510 51539 35931 21006 48341 52928 10998 41373 33142]
epoch=1,  batch_mask=[40882 13862 24650 54847  1738 11193 31861 49329 37458  1891]
epoc

epoch=1,  batch_mask=[11905 30175 36692  4584 23153 19990 35044 19822 55481 17868]
epoch=1,  batch_mask=[11692 37509 39611 52109 22271 36379 58949 29664 44013 39664]
epoch=1,  batch_mask=[19537 25735   590 51296 14367 41276 24604 14958 42517  3030]
epoch=1,  batch_mask=[48997  2915 53661 42791 39598 16477 40949 44343  5821  9185]
epoch=1,  batch_mask=[56471  4333 34751 20131 42029 11938 51843 51265 28676  8732]
epoch=1,  batch_mask=[48491  3830  1128 32096 19876 41475 57885    73 38544 45385]
epoch=1,  batch_mask=[ 8672 55675 29814 20185 25442  5742 33818 38592 33989 26064]
epoch=1,  batch_mask=[20446 54203  2701 54172 59943 55996   629 46573  7085 33941]
epoch=1,  batch_mask=[47832 55499 14059 55074 18438 16202 18152 24939 58680 54149]
epoch=1,  batch_mask=[24593 16642 57246 53981 31292  3802 49615 35479 21764 46072]
epoch=1,  batch_mask=[40818 26666 47676 23059 11235 46697 47770 51781   211 49257]
epoch=1,  batch_mask=[48570 47472 21422 25947 43945   484 51462 27251 58695 52178]
epoc

epoch=2,  batch_mask=[53626 36706 54794 50432 33128 28258 34708 59738  8587 50780]
epoch=2,  batch_mask=[57332 10911  5553 25803 46067 15830 59677 21212  4448  5502]
epoch=2,  batch_mask=[33493 20800 35221 25449  5015  7798 58714 24652  8117  7599]
epoch=2,  batch_mask=[24160 59076 53440 31744 35511 18563 37098 43446 19544 47859]
epoch=2,  batch_mask=[47073 35175 46389 13074 31464 55529 52041 34482 14234 58128]
epoch=2,  batch_mask=[49819 52540 50991 29820 53751  2711 16284  7605 34113 13023]
epoch=2,  batch_mask=[28098 52909 12507 13572 13992 59781 32747 54984 32565 45815]
epoch=2,  batch_mask=[ 6182 11744 41072 54346 43645 28205  3460 37106 44491 35177]
epoch=2,  batch_mask=[32855 20941 42548 22286  9065   399 25837 20583  9669 12415]
epoch=2,  batch_mask=[45928  3547 29668 26198 14965 53137 13274 47620 28886 54991]
epoch=2,  batch_mask=[ 2311  2068 26943 50927 48715 45179 52549 48494 34230 27523]
epoch=2,  batch_mask=[ 9283 11900  7643 38907 39503 11719 25377 19724 52329 57771]
epoc

...

epoch=7,  batch_mask=[ 7755 25924 44913 47036 23807 52923 50195 11312 40477 52963]
epoch=7,  batch_mask=[ 1344 13755 45267 46501 20197 22793 27359 21602 11742 55725]
epoch=7,  batch_mask=[37419 43170 22084 21832  2907 53303 19705 44903 44325 18744]
epoch=7,  batch_mask=[29902 52437   327 55242 46714 52005 30504 32047 31391 12994]
epoch=7,  batch_mask=[20125  9720 58011 33453 18682   670 53159 22342 30600  8139]
epoch=7,  batch_mask=[ 7190 42064 35549  5609  6918 53630 12852 52158 28729 28260]
epoch=7,  batch_mask=[ 5083 35823  9713  8114 42864 54749 27138 22159 16096  7432]
epoch=7,  batch_mask=[28909 15547  1226 20333 39737 28628 33987 16116  1394 19542]
epoch=7,  batch_mask=[38752 11140 34178 56874 20628 40659 40988 59963 37207 34730]
epoch=7,  batch_mask=[31753  4263 29944 38623 18718 48236 52954 53136 38509   600]
epoch=7,  batch_mask=[31545 53405 18570 19763  8761 14924 16014 10299 35503  1268]
epoch=7,  batch_mask=[11946 37233 49696 27576 53847 33381  7849  9943   755 49135]
epoc

epoch=8,  batch_mask=[45305 10690  7910 29010 52523  5684 37596  4026 32965 47553]
epoch=8,  batch_mask=[20579 34769 56246 16436 20312  1704 23434   224 37629 36434]
epoch=8,  batch_mask=[39783 40506 43717 11525 33908 35092 25441 32005 53631 44484]
epoch=8,  batch_mask=[34573 35537 36464 21260 38850 19781 34204  9942 32929 23854]
epoch=8,  batch_mask=[ 8978 37846 17777 39970 39791 25763 28254 55311 20313 44029]
epoch=8,  batch_mask=[12209  1291 25585 55297 41011 53011 47527 55100 11231 20490]
epoch=8,  batch_mask=[12097  7585 21253 54163 13535 49139 23660 31682 29719 15745]
epoch=8,  batch_mask=[34143 33884 27568  9796 30059 57192 46617  6024 45443 50535]
epoch=8,  batch_mask=[53345 47085 47727 29732 32576 20051 17711 23481 39712 27600]
epoch=8,  batch_mask=[59164 54002 16108 29947 10543 13026 58206 32845  5839 43009]
epoch=8,  batch_mask=[25099 38168 47790 10037  5767 59909 41598 10882 52627 55274]
epoch=8,  batch_mask=[17624 47425 33172 41834 28312 17122 47275 57379 39423 37728]
epoc

epoch=8,  batch_mask=[37215 50042  9758 30377 17677 41175 41211 23388 41447  5888]
epoch=8,  batch_mask=[15407 44389   530  1759  2003  2585 13045 52805 40406   422]
epoch=8,  batch_mask=[10706 36562  2556  5853 34549 40201  4935 30672 45991 52515]
epoch=8,  batch_mask=[ 1254 45790 11063 25877  9122 23160 48285  5127 51652 43444]
epoch=8,  batch_mask=[ 7951  8571  3904   943 46159 29759 25627 52232 56840 44572]
epoch=8,  batch_mask=[22459 30222 52719 31363 25843 35510 55949 20678 32216 12458]
epoch=8,  batch_mask=[  611 21483 14546 27388 41959 56466 28525  7427 48474 47673]
epoch=8,  batch_mask=[ 4746 56053 41848 33566 24440 37412 33770 27341   231 24588]
epoch=8,  batch_mask=[  391 16587 42570  1670 21336 59266  1387 55372 12501  8885]
epoch=8,  batch_mask=[51668 20469 11275 50027  6194 24351 54750 55873 54851  1991]
epoch=8,  batch_mask=[55089 28691 33268 41546 26358 25020 26532 22512 43030 17436]
epoch=8,  batch_mask=[22860 47832 52948 50588 11913 35734  3084 23083 35867 34008]
epoc

epoch=9,  batch_mask=[10209  1857  1897  8214 25908 12648 39243  2503 33935 59836]
epoch=9,  batch_mask=[42635 26856 39286 27464 24962 38897 15206 53646 59133 44573]
epoch=9,  batch_mask=[  717 12215 53261  4671 44027 55960 18628 19636 23440    19]
epoch=9,  batch_mask=[11032  5846 29913 53812 51522 35342 29254 41622 36482 21622]
epoch=9,  batch_mask=[ 4791 43569 27635 57610  6597 37298 17632 19609  1275   206]
epoch=9,  batch_mask=[57930 18325 44934 44933 50733 59120 43340 13041 57557 40639]
epoch=9,  batch_mask=[25457 16947 39295 56464 57330 39056 36064 13321 37901 49412]
epoch=9,  batch_mask=[57335 56937  6987  3445  2768  8342 53037 29412 24590 21446]
epoch=9,  batch_mask=[37281  3664 22300 57148 58564   220  6915 29384 31300 19014]
epoch=9,  batch_mask=[11954 33771   752 27604 38322 17066 20167 31647  1495 21359]
epoch=9,  batch_mask=[ 6682 26534 31261 24949 21234 47339 55031 46614 14307 34458]
epoch=9,  batch_mask=[ 4661 39107 24003 57131  8600 35398 51022 39790 37502  7207]
epoc

epoch=9,  batch_mask=[16783 31101 20283 57991 59299 11950 10374 59743 14839 40227]
epoch=9,  batch_mask=[13663 30326 55205 26724  9636 33738 51588 55109 42658 24037]
epoch=9,  batch_mask=[ 8003 26398 18869 51622 57106 48240 36475 45324 41776 57229]
epoch=9,  batch_mask=[55453 43927 52919 28529 49854 54188 26734 43227 13392 42663]
epoch=9,  batch_mask=[27305 52155 13414 46946 16287 59982 59923 44204 21648 59360]
epoch=9,  batch_mask=[15607 26387 53997 50627 34396  5132 53484 15416  5668 43532]
epoch=9,  batch_mask=[58277  8625  9156 25664 43813 53228 56707 25918 11030 45546]
epoch=9,  batch_mask=[16954 57818 35794 49480  8228 50195 32424  2296 58824 50396]
epoch=9,  batch_mask=[36079 19412 53846 16875 42007 15651 45446 20358 56902 11653]
epoch=9,  batch_mask=[32872 40711 25031  8681 19610 48987 15199 36237 57015 40898]
epoch=9,  batch_mask=[38699  1047 15319 19167   397 48935 16255 47037 28510 53320]
epoch=9,  batch_mask=[25000  2474 18543 41040 42173   438 45208 59320 44552 51168]
epoc